# PÖFF's movie info

In [9]:
import pandas as pd
import os
from datetime import date
from os import listdir
from os.path import isfile, join
import requests
import re
import imdb

import qgrid
#jupyter nbextension enable --py --sys-prefix qgrid
#jupyter nbextension enable --py --sys-prefix widgetsnbextension

def createP6ffMovieDataFile():

    p6ff_url = "https://poff.ee/en/search_film/"

    # Get movie titles
    movies_html = requests.get(p6ff_url)
    year_html = iter(movies_html.text.split("\n"))
    movies = []
    for i, line in enumerate(year_html):
        # find line wih movie titels
        if (line.find('<div class="card_film_title">') != -1):

            movie_titles = re.findall('<h3 class="film_title_locale dark_1" id="title">([a-zA-Z0-9 \-.:/]+)</h3>',line)

            #print (sorted(movie_titles))

    # Get movie data
    i = imdb.IMDb()
    movies = {}
    movies_data = {"title": [],
                  "rating": [],
                  "votes": [],
                  "year": [],
                  "genres": [],
                  "cover url": [],
                  "imdbID": [],
                  "imdbLink": [],
                  "imdb": []}
    for j,title in enumerate(sorted(movie_titles)):                    
        print ("\t",title)
        movies[title] = {}
        best_match = i.search_movie(title)

        movies_data["title"].append(title)


        #if no matches
        if len(best_match) == 0: 
            movies_data["rating"].append(0)
            movies_data["votes"].append(0)
            movies_data["year"].append(0)
            movies_data["genres"].append("")
            movies_data["cover url"].append("")
            movies_data["imdbID"].append("")
            movies_data["imdbLink"].append("")
            movies_data["imdb"].append("") 
            continue

        best_match =i.search_movie(title)[0] #first match is chosen
        i.update(best_match)
        movies[title]["imdb"] = best_match
        movies_data["imdb"].append(best_match)

        if "rating" in best_match.keys():
            movies_data["rating"].append(best_match["rating"])
        else:
            movies_data["rating"].append("");

        if "year" in best_match.keys():
            movies_data["year"].append(best_match["year"])
        else:
            movies_data["year"].append("");

        if "genres" in best_match.keys():
            movies_data["genres"].append(";".join(best_match["genres"]))
        else:
            movies_data["genres"].append("");

        if "votes" in best_match.keys():
            movies_data["votes"].append(best_match["votes"])
        else:
            movies_data["votes"].append("");

        if "cover url" in best_match.keys():
            movies_data["cover url"].append(best_match["cover url"])
        else:
            movies_data["cover url"].append("");

        if "imdbID" in best_match.keys():
            movies_data["imdbID"].append(best_match["imdbID"])
            movies_data["imdbLink"].append('<a href="http://www.imdb.com/title/tt' + best_match["imdbID"] + '/">Link</a>')
        else:
            movies_data["imdbID"].append("");
            movies_data["imdbLink"].append("");

    #print(" - ".join([title,str(movies[title]["rating"]),str(movies[title]["votes"]), str(movies[title]["year"]), ";".join(movies[title]["genres"])]))    

    # write data to file
    df = pd.DataFrame(movies_data, columns= ['title', 'rating', 'votes', 'year', 'genres' , 'imdbLink'])

    today = date.today()
    fileName = os.path.join("data","movies_data.csv")
    newFileName = os.path.join("data","movies_data_" + str(today) + ".csv")
    df.to_csv(newFileName, index = False, header=True)

# get all data files
onlyfiles = [f for f in listdir("data") if isfile(join("data", f))]

# if data files are missing create one
if (len(onlyfiles) == 0):
    createP6ffMovieDataFile()

# last imdb data file
lastFile = onlyfiles[len(onlyfiles)-1]

# extract data
df = pd.read_csv(os.path.join("data",lastFile)) 

# display data
df = df.sort_values("rating", ascending = False)
df

,title,rating,votes,year,genres,imdbLink
106,Love Song 1980,9.4,10.0,2020.0,Drama,"<a href=""http://www.imdb.com/title/tt10097470/..."
81,Holy Father,9.2,45.0,2020.0,Documentary,"<a href=""http://www.imdb.com/title/tt12833692/..."
141,Remy and Juliyat,9.2,6.0,2019.0,Drama,"<a href=""http://www.imdb.com/title/tt9717124/""..."
143,Russian Death,8.8,128.0,2019.0,Documentary;Horror,"<a href=""http://www.imdb.com/title/tt10544422/..."
134,Psycho,8.5,595752.0,1960.0,Horror;Mystery;Thriller,"<a href=""http://www.imdb.com/title/tt0054215/""..."
49,Cry Wolf,8.4,99.0,2020.0,Crime;Drama;Mystery,"<a href=""http://www.imdb.com/title/tt11823102/..."
31,Blizzard of Souls,8.4,1459.0,2019.0,Drama;History;War,"<a href=""http://www.imdb.com/title/tt6084202/""..."
13,Anton,8.4,29.0,2019.0,Drama,"<a href=""http://www.imdb.com/title/tt6872792/""..."
60,Everything: The Real Thing Story,8.3,64.0,2019.0,Documentary,"<a href=""http://www.imdb.com/title/tt11046788/..."
51,Davos,8.3,7.0,2020.0,Documentary,"<a href=""http://www.imdb.com/title/tt12306916/..."


In [10]:
qgrid_widget = qgrid.show_grid(df)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [6]:
## if last data files is not from today it will creat new one

today = date.today()
lastFileDate = lastFile.split(".")[0].split("_")[2]
if (str(today) != lastFileDate):
    print(lastFileDate)
    print(str(today))
    createP6ffMovieDataFile()

2020-11-16
2020-11-17
	 101 Estonian Films: Animation
	 25 Years of Innocence. The Case of Tomek Komenda
	 50 or Two Whales Meet on the Beach
	 A Distant Place
	 A Man Who Never Was
	 A Yellow Animal
	 Abulele
	 Ammonite
	 An Ordinary Woman 2
	 And Tomorrow the Entire World
	 Angel
	 Animated Journeys
	 Animated Realities
	 Anton
	 Armugan
	 Art Paul of Playboy: The Man Behind The Bunny
	 As Far as I Know
	 Ascent of a Woman - Live Action Documentaries 2
	 Baby
	 Bad Tales
	 Barefoot: The Mark Baumer Story
	 Basic Care Unit
	 Beasts
	 Because of My Body
	 Bedridden
	 Beginning
	 Berlin Alexanderplatz
	 Big vs Small
	 Bitter Love
	 Black Light
	 Black Milk
	 Blizzard of Souls
	 Boys Boys Boys - Live Action Documentaries 1
	 Brave New Worlds: Scifi and Fantasy Shorts
	 Bula
	 Caged Birds
	 Call Me Blackbird
	 Catwalk: From Glada Hudik to New York
	 Caught in the Net
	 Charlatan
	 Chicks
	 City So Real
	 Cocoon
	 Colourful Dreams
	 Come True
	 Conference
	 Cortex
	 County Lines
	 Courtroo